In [49]:
#Imports
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from datetime import date

#Define Google API Scopes
#If modifying these scopes, delete the file token.json.
#Scope defines level of access to resources required by your app like level of access to data owned by the users.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly',
          'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/presentations']

#Credentials - Reads in credentials From token.json stored in same folder as script
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=8080)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

#API Call Authorization
google_slides = build('slides', 'v1', credentials=creds)
google_drive = build('drive', 'v3', credentials=creds)
google_sheets = build('sheets', 'v4', credentials=creds)


#Get Today's Date - This will be used in naming new files and finding sheets from the current day
todays_date = date.today().strftime("%m%d%Y")
presentation_date = date.today().strftime("%-m/%-d/%Y")

def sheet_check():
    
    #Search for a Google Sheet in the drive saved with the naming convention 'Metrics_' + todays_date
    sheet_name = 'Metrics_' + todays_date
    sheets_response = google_drive.files().list(q="name = '{}'".format(sheet_name)).execute()

    #If there isn't a Google Sheet available in the drive from the current day then print'No New Data Available Today'
    #Otherwise
    #Run the slide_check() fucntion 
    if sheets_response.get('files') == []:
        print('No New Data Available Today')
    else:
        slide_check()

def slide_check():
    
    #Search for a Google Slide in the drive saved with the naming convention 'Presentation_' + todays_date
    presentation_name = 'Presentation_' + todays_date
    slides_response = google_drive.files().list(q="name = '{}'".format(presentation_name)).execute()

    #If there isn't a Google Slide available in the drive from the current day then run slide_creation()
    #Otherwise
    #If there is a Google Slide available in the drive from the current day, delete it and run slide_creation()
    if len(slides_response.get('files', [])) == 0:
        slide_creation()
    else:
        for file in slides_response.get('files', []):
            google_drive.files().delete(fileId=file.get('id')).execute()
        else:
            slide_creation()            
            
def slide_creation():

    #Presentation ID of the Google Slide we are using as our template
    #This will presentation will be copied and the copy will have a new and different ID
    presentation_id = '16ffoRD-e7pESFxj6LyZjDF7mGYKlSbaoXLm94GQPESE'

    #When a copy of the presentation is made. Call it Presentation_' + todays_date' and place it in the Google drive folder whose ID is define by 'parents'
    body = {
    'name': 'Presentation_' + todays_date,
    'parents' : ['155l-i-Es3-noh5J7uwONO3IyNDerK_K1']   
    }

    #Create a copy of the presentation 
    drive_response = google_drive.files().copy(fileId=presentation_id, body=body).execute()
    
    #Get presentation ID of copied presentation 
    presentation_copy_id = drive_response.get('id')
    
    #Replace {{Date}} on the presentation with the current day's date
    reqs = [(
    {'replaceAllText': {
        'containsText': {'text': '{{Date}}'},
        'replaceText': presentation_date
    }})]
    
    #Make changes/updates to presentation. Anything set to reqs or reqs.append will be updated on the presentation
    google_slides.presentations().batchUpdate(body={'requests': reqs},presentationId=presentation_copy_id).execute()
    
    #Search for a Google Sheet in the drive saved with the naming convention 'Metrics_' + todays_date
    sheet_name = 'Metrics_' + todays_date
    sheets_response = google_drive.files().list(q="name = '{}'".format(sheet_name)).execute()
    
    #Get the ID of the sheet with name - Metrics_' + todays_date
    sheetID = str(sheets_response.get('files')[0]['id'])
    
    #Get all values on 'Sheet1' of the Google Sheet
    orders = google_sheets.spreadsheets().values().get(range='Sheet1',spreadsheetId=sheetID).execute().get('values')
    
    #Create a table on the second slide of the presenation with len(orders) rows and len(orders[0] columns
    reqs = [
    {'createTable': {
        'elementProperties': {'pageObjectId': 'gd80e96dd8b_0_0'},
        'rows': len(orders),
        'columns': len(orders[0])},
    }]

    #Store data about the objects in the presentation
    rsp = google_slides.presentations().batchUpdate(body={'requests': reqs}, presentationId=presentation_copy_id).execute().get('replies')
    
    #Get ID of the table on the second slide
    tableID = rsp[0]['createTable']['objectId']
    
    #Add values from Sheet1 to the table on slide 2
    reqs = [
    {'insertText': {
        'objectId': tableID,
        'cellLocation': {'rowIndex': i, 'columnIndex': j},
        'text': str(data),
    }} for i, order in enumerate(orders) for j, data in enumerate(order)]
    
    #Make changes/updates to presentation. Anything set to reqs or reqs.append will be updated on the presentation
    google_slides.presentations().batchUpdate(body={'requests': reqs},presentationId=presentation_copy_id).execute() 
     
    #Bold the header in column 2 on of the table on slide 2      
    reqs =  [
    {
      "updateTextStyle": {
        "objectId": tableID,
        "cellLocation": {
          "rowIndex": 0,
          "columnIndex": 1
        },
        "style": {
          "foregroundColor": {
            "opaqueColor": {
              "rgbColor": {
                "red": 0.0,
                "green": 0.0,
                "blue": 0.0
              }
            }
          },
          "bold": True
        },
        "textRange": {
          "type": "ALL"
        },
        "fields": "foregroundColor,bold,fontFamily,fontSize"
      }}]
    
    #Bold the header in column 2 on of the table on slide 2   
    reqs.append(
    {
      "updateTextStyle": {
        "objectId": tableID,
        "cellLocation": {
          "rowIndex": 0,
          "columnIndex": 0
        },
        "style": {
          "foregroundColor": {
            "opaqueColor": {
              "rgbColor": {
                "red": 0.0,
                "green": 0.0,
                "blue": 0.0
              }
            }
          },
          "bold": True
        },
        "textRange": {
          "type": "ALL"
        },
        "fields": "foregroundColor,bold,fontFamily,fontSize"
      }})
    
    #Make changes/updates to presentation. Anything set to reqs or reqs.append will be updated on the presentation  
    google_slides.presentations().batchUpdate(body={'requests': reqs},presentationId=presentation_copy_id).execute() 
            
    #Get the product name and number of products sold of the most sold product
    product = max(orders)[0]
    num_of_product = max(orders)[1]

    #Replace {{product}} on the second slide with the name of the most sold product
    reqs = [
    {'replaceAllText': {
        'containsText': {'text': '{{product}}'},
        'replaceText': product
    }}]
    
    #Replace {{num_of_product}} on the second slide with the number of units of the most sold product
    reqs.append(
    {'replaceAllText': {
        'containsText': {'text': '{{num_of_product}}'},
        'replaceText': num_of_product
    }}) 
    
    #Add the pie chart from Sheet1 to Slide 3 of the presentation. Adjust the size and position accordingly
    reqs.append({'createSheetsChart': {
    'spreadsheetId': sheetID,
    'chartId': '1870828732',
    'elementProperties': {
        'pageObjectId': 'gd80e96dd8b_0_5',
        'size': {
            'height': {'magnitude': 6000, 'unit': 'EMU'},
            'width':  {'magnitude': 9000, 'unit': 'EMU'}
        },
        'transform': {
            'scaleX': 696.6157,
            'scaleY': 601.3921,
            'translateX': 1200000,
            'translateY': 1200000,
            'unit': 'EMU',
        },
    },
}})
    
    #Make changes/updates to presentation. Anything set to reqs or reqs.append will be updated on the presentation
    google_slides.presentations().batchUpdate(body={'requests': reqs},presentationId=presentation_copy_id).execute()
    
#Run sheet_check()
sheet_check()